In [0]:
!pip install langdetect
!pip install tweet-preprocessor
!pip install nltk

In [0]:
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
emoticons = emoticons_happy.union(emoticons_sad)

In [0]:
pd.options.mode.chained_assignment = None

import pandas as pd
import langdetect
import preprocessor as pre
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

words = set(nltk.corpus.words.words())

#load data
data = pd.read_csv("/content/drive/My Drive/00.csv")
df = data.loc[:,['entities.hashtags.0.text','entities.hashtags.1.text','retweet_count','text']]

#remove nan rows
df = df[pd.notnull(df['text'])]

def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
    
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    tweet = emoji_pattern.sub(r'', tweet)
    
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
    for w in word_tokens:
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)

df = df[pd.notnull(df['text'])]

for index,row in df.iterrows():
  list2 = []
  text = " ".join(w for w in nltk.wordpunct_tokenize(row['text']) \
         if w.lower() in words or not w.isalpha())
  text = pre.clean(str(text))
  clean_text = clean_tweets(text)
  list1 = list()
  p = re.compile("[A-Z]|[a-z]")
  for x in clean_text:
    if(x == ' '):
      st = ''.join(list1)
      list2.append(st)
      list1 = []
    if(p.match(x)):
      list1.append(x)
  st = ''.join(list1)
  list2.append(st)
  list2 = list(filter(None,list2)) 
  st1 = ' '.join(list2)
  df['text'][index] = st1
  df['text'][index] = re.sub(r'http\S+', '', str(df['text'][index]))
  
for index,row in df.iterrows():
  if(df.loc[index]['text'] == ''):
    df.drop([index], inplace = True)
print(df['text'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
0                                        Maxedout FKMnrKK
3                                                       I
4       SO EXCITED FOR TONIGHT MY DAD AROUND WAITING F...
6                                      Me pagan karate de
7                             YES x I tell much important
8                                        Mi define mi soy
9                  Ni en para la sale arriba de la bikini
10                          nu met met thee ens half back
12                                  Can go launch already
13              Top Best Gangster Past struggling make It
14      rcooley Cheap Color Change Test Could Revoluti...
15                   

In [0]:
df.to_csv("preprocessed1.csv", sep='\t')